In [1]:
#########################################
# Test functions and non-permanent code #
# Author: Jarrett Reimers               #

# Keep as a reference, do not edit      #
#########################################

import pandas as pd
import numpy as np
import os
import datetime

import query
from model import Model
import parameter
import plotly.express as px
import simplejson, urllib
import plotly.graph_objects as go

import matplotlib.pyplot as plt

from matplotlib import pyplot
import seaborn as sns

if not os.path.exists("images"):
    os.mkdir("images")

In [2]:
df = parameter.get_station_data(True)


In [4]:
df

,name,capacity,station_id,short_name,region_id,lat,lon,num_bikes_available,num_bikes_disabled,num_docks_available,num_docks_disabled
0,Whitehall St & Bridge St,1,c1a4d909-0a00-475a-8e82-18ed13a4eb01,4962.02,71,40.703662,-74.013181,0,0,0,0
1,Broadway & E 19 St,65,40043e8a-6ef6-428d-b585-a7c23cdb06d4,6098.12,71,40.738661,-73.989873,0,0,0,0
2,W Broadway & Watts St,51,7d5fa1f0-4069-4d5d-b735-5d500597394a,5569.07,71,40.723230,-74.003140,0,0,0,0
3,Cauldwell Ave & E 161 St,18,493850bd-125b-49f5-932b-bf999504fb06,7952.02,71,40.821880,-73.907521,0,0,0,0
4,Borden Ave & Review Ave,19,6d766daf-9298-4059-a329-1818582d495f,6001.01,71,40.738663,-73.940938,14,0,5,0
...,...,...,...,...,...,...,...,...,...,...,...
2209,Columbus Drive,14,66dddc08-0aca-11e7-82f6-3863bb44ef7c,JC014,70,40.718355,-74.038914,10,3,1,0
2210,Warren St,22,66dd5a42-0aca-11e7-82f6-3863bb44ef7c,JC006,70,40.721124,-74.038051,17,4,1,0
2211,2 St & Park Ave,15,b6a13f2b-deb4-4f4d-9d67-7d23bee27c88,HB608,311,40.739153,-74.033082,8,2,5,0
2212,Jersey & 6th St,14,66ddd93e-0aca-11e7-82f6-3863bb44ef7c,JC027,70,40.725289,-74.045572,0,0,14,0


In [2]:
start_date = query.get_datetime(2023, 5, 1, 0, 0, 0)
end_date = query.get_datetime(2023, 5, 30, 0, 0, 0)
whitelist_stations = ['E 7 St & Ave B',
                      'Cooper Square & Astor Pl',
                      'E 7 St & Ave C',
                      'Ave A & E 14 St',
                      'W 21 St & 6 Ave']

parameter.avg_travel_time(whitelist_stations[0], whitelist_stations[1], start_date, end_date)
# parameter.get_transition('W 21 St & 6 Ave', start_date=start_date, end_date=end_date, tph=1)

Timedelta('0 days 00:07:10.272317403')

In [7]:
tph = 2
date = datetime.timedelta(hours=0.1)
date

datetime.timedelta(seconds=360)

In [ ]:
start_date = query.get_datetime(2023, 5, 1, 0, 0, 0)
end_date = query.get_datetime(2023, 6, 30, 0, 0, 0)
parameter.get_rate('2 Ave & E 29 St',start_date=start_date, end_date=end_date, length=datetime.timedelta(hours=1), weekday=True)

In [11]:
data = pd.read_csv("data/2023/by_station/Cooper Square & Astor Pl.csv", low_memory=False, index_col=0)
data = query.make_datetime(data)
stations = data['end_station_name'].unique()
good_stations = []
for station in stations:
    num = len(data.loc[data['end_station_name'] == station])
    if num/len(data) > 0.008:
        good_stations.append(station)
print(good_stations)

In [ ]:
station_list = query.get_stations(data)
station_id = 324 
station_data = query.select_start_station(data, station_id)
stations = Model()
start_time = query.get_datetime(2015, 5, 1, 0, 0, 0)

freq = []
for day in range(0, 60):
  stations.clear_data()
  window = datetime.timedelta(days=1)
  end_time = start_time + window
  window_data = query.select_time(station_data, start_time, end_time)
  stations.add_data(window_data)

  # print("Total trips started on day", day)
  # if station_id in stations.stations_count:
  #   print(stations.stations_count[station_id])
  # else:
  #   print(0)
  
  daily = []
  for hour in range(0, 24):
    stations.clear_data()
    window = datetime.timedelta(hours=1)
    end_time = start_time + window
    window_data = query.select_time(station_data, start_time, end_time)
    stations.add_data(window_data)

    if station_id in stations.stations_count:
      # print(hour, stations.stations_count[station_id])
      daily.append(stations.stations_count[station_id])
    else:
      # print(hour, 0)
      daily.append(0)
      
    start_time = end_time
  freq.append(daily)
  start_time = end_time


In [ ]:
start_time = query.get_datetime(2023, 4, 1, 0, 0, 0)
end_time = query.get_datetime(2023, 7, 1, 0, 0, 0)
interval = 0.5

freqs = []
while start_time < end_time:
    freq = []
    if start_time.weekday() > 5:
        start_time += datetime.timedelta(days=1)
        continue
    daily_rides = len(query.select_time(data, start_time, start_time+datetime.timedelta(days=1)))
    if daily_rides < 150 or daily_rides > 250:
        start_time += datetime.timedelta(days=1)
        continue   
    for hour in range(int(24/interval)):
        stop_time = start_time + datetime.timedelta(hours=interval)
        rides = len(query.select_time(data, start_time, stop_time))
        freq.append(rides)
        start_time = stop_time
    freqs.append(freq)

In [ ]:
df = pd.DataFrame(freqs)
# p = 0.73
plt.plot(df.var() - df.mean())

In [ ]:
p = 0.73295
(df.var() - df.mean()/p).sum()

In [ ]:
col = 27
plt.hist(df[col].T, bins=df[col].T.max())

In [ ]:
plt.hist(df.T.sum(), bins=30)

In [ ]:
for q in df.quantile(0.1):
    trimmed_df = df.loc[df[col] > q]

In [ ]:
# stations = Stations()
# stations.add_data(query.select_time(data, start_time, end_time))
#   
# pyplot.plot(range(len(stations.stations_count)), stations.stations_count.values())

In [ ]:
maximum = 0
max_station = 0
for station in stations.stations_count:
  if stations.stations_count[station] > maximum:
    max_station = station
    maximum = stations.stations_count[station]
print("The maximum is at station:", max_station, ". With", maximum, "bikes at end of period")

In [ ]:
minimum = 0
min_station = 0
for station in stations.stations_count:
  if stations.stations_count[station] < minimum:
    min_station = station
    minimum = stations.stations_count[station]
print("The minimum is at station:", min_station, ". With", minimum, "bikes at end of period")

In [ ]:
windows = [0.25, 0.5, 1]

start_time = query.get_datetime(2015, 5, 1, 0, 0, 0)
stop_time = query.get_datetime(2015, 5, 31, 0, 0, 0)
weekdays_and_ends = parameter.get_weekdays_and_weekends(start_time, stop_time)
weekdays = weekdays_and_ends[0]
weekends = weekdays_and_ends[1]
stations = (data['start station id']).unique()

daily_trips = []
for start_time in weekdays:
  end_time = start_time + datetime.timedelta(days=1)
  daily_trips.append(len(query.select_time(data, start_time, end_time)))
  start_time += datetime.timedelta(days=1)
# throughput = pd.cut(daily_trips, 3, labels=range(3))

for hours in windows:
  # for station_id in stations[:5]:
    weekday_freqs = [] 
    station_id = 537
    station_df = query.select_start_station(data, station_id)
    
    for start_time in weekdays:
      freq = []
      stop_time = start_time+datetime.timedelta(days=1)
      time_df = query.select_time(station_df, start_time, stop_time).reset_index(drop=True)
      chop_df = parameter.cut(time_df, start_time, stop_time, datetime.timedelta(hours=hours))
      for i in range(len(chop_df)):
        freq.append(len(chop_df[i]))
      weekday_freqs.append(freq)
    
    rows = []
    i = 0
    for daily in weekday_freqs:
      hour = 0
      for hourly in daily:
        rows.append([hourly, hour, daily_trips[i]])
        hour += 1
      i+=1 
    df = pd.DataFrame(rows, columns=['trips', 'hours', 'Total throughput'])
    
    trip_data = pd.DataFrame(weekday_freqs)
    rate = trip_data.sum()/len(trip_data.T.iloc[0])
    pred = np.random.poisson(rate)
    
    fig = px.scatter(df, x='hours', y='trips', color='Total throughput', template='plotly',
                     width=1200, height=500, labels={'hours': 'Half-hours', 'trips': 'Departures', 'Total throughput': 'Total throughput'}, title="Daily departure breakdown (May, weekday only)")
    fig.add_trace(go.Scatter(x=np.array(range(int(24/hours))), y=np.array(pred),
                        mode='lines',
                        name=''))
    
    if not os.path.exists(f"images/{station_id}/"):
      os.mkdir(f"images/{station_id}/")
    fig.write_image(f'images/{station_id}/{hours}_test.png')

In [ ]:
station_data = query.select_start_station(data, 537)
start_time = query.get_datetime(2015, 5, 1, 0, 0, 0)
end_time = query.get_datetime(2015, 5, 31, 0, 0, 0)

slide_data = []
while start_time <= end_time:
    stop_time = start_time+datetime.timedelta(days=1)
    time_df = query.select_time(station_data, start_time, stop_time).reset_index(drop=True)
    slide_data.append(len(time_df))
    start_time = stop_time
    

In [ ]:
start_time = query.get_datetime(2015, 5, 1, 0, 0, 0)
end_time = query.get_datetime(2015, 5, 31, 0, 0, 0)
station_data = query.select_time(data, start_time, end_time).reset_index(drop=True)
weekdays_and_ends = parameter.get_weekdays_and_weekends(start_time, end_time)
weekdays = weekdays_and_ends[0]
slide_freqs = [] 

for start_time in weekdays:
    freq = []
    stop_time = start_time+datetime.timedelta(days=1)
    time_df = query.select_time(station_data, start_time, stop_time).reset_index(drop=True)
    chop_df = parameter.cut(time_df, start_time, stop_time, datetime.timedelta(hours=0.5))
    for i in range(len(chop_df)):
      freq.append(len(chop_df[i]))
    slide_freqs.append(freq)
slide_freqs = pd.DataFrame(slide_freqs)

In [ ]:
path = 'data/2023/by_station/'
for file in os.listdir(path)[::20]:
    data = pd.read_csv(path+file)
    data = query.make_datetime(data)
    start_time = query.get_datetime(2023, 5, 1, 0, 0, 0)
    end_time = query.get_datetime(2023, 9, 30, 0, 0, 0)
    station_data = query.select_time(data, start_time, end_time)
    weekdays_and_ends = parameter.get_weekdays_and_weekends(start_time, end_time)
    weekdays = weekdays_and_ends[0]
    freq = []
    hour = 17
    window = 2
    
    for day in weekdays:
        start_time = day + datetime.timedelta(hours=hour)
        stop_time = start_time + datetime.timedelta(hours=window)
        time_df = query.select_time(station_data, start_time, stop_time).reset_index(drop=True)
        freq.append(len(time_df))
        
    df = pd.DataFrame(freq)
    if df[0].max() == 0:
        print("No departures for", file)
        continue
    plt.hist(df[0], bins=int(df[0].max()))        
    plt.savefig(f'images/histograms/{file[:-4]}.png')
    plt.close()
    

In [ ]:
start_date = query.get_datetime(2023, 5, 1, 0, 0, 0)
end_date = query.get_datetime(2023, 6, 30, 0, 0, 0)
parameter.get_rate('2 Ave & E 29 St',start_date=start_date, end_date=end_date, length=datetime.timedelta(hours=1))

['4 Ave & E 12 St', '1 Ave & E 6 St', 'E 7 St & Ave B', 'Cooper Square & Astor Pl', 'St Marks Pl & 1 Ave', 'E 5 St & Ave A', 'E 7 St & Ave C', 'E 5 St & Avenue C', 'E 4 St & Ave B', 'E 7 St & Avenue A', 'E 17 St & Broadway', 'Ave A & E 14 St', 'Broadway & E 14 St', 'E 6 St & Avenue B', 'E 6 St & 2 Ave', 'W 21 St & 6 Ave']


In [ ]:
data = pd.read_csv('data/2023/by_station/Riverside Dr & W 78 St.csv', low_memory=False, index_col=0)
data.dropna(axis="rows", inplace=True)
data = query.make_datetime(data)

In [ ]:
start_time = query.get_datetime(2023, 5, 1, 0, 0, 0)
end_time = query.get_datetime(2023, 8, 30, 0, 0, 0)
length = datetime.timedelta(hours=1)

whitelist_stations = ['4 Ave & E 12 St',
 '1 Ave & E 6 St',
 'E 7 St & Ave B',
 'Cooper Square & Astor Pl',
 'St Marks Pl & 1 Ave',
 'E 5 St & Ave A',
 'E 7 St & Ave C',
 'E 4 St & Ave B',
 'E 7 St & Avenue A',
 'Ave A & E 14 St']

cut_df = query.select_time(data, start_time, end_time)
stations_transition_by_hour = {hour: {} for hour in range(24)}
trip_count = {i: 0 for i in range(24)}

weekdays = parameter.get_weekdays_and_weekends(start_time, end_time)[0]
total_unique_stations = len(cut_df['end_station_name'].unique())
for day in weekdays:
    end_time = day + datetime.timedelta(days=1)
    cut_list = parameter.cut(cut_df, day, end_time, length)
    for i in range(len(cut_list)):
        total_trips = 0
        for station in cut_list[i]['end_station_name'].unique():
            if station not in whitelist_stations:
                continue
            station_trips = len(cut_list[i].loc[cut_list[i]['end_station_name'] == station])
            total_trips += station_trips
            if station in stations_transition_by_hour[i]:
                stations_transition_by_hour[i][station] += station_trips
            else:
                stations_transition_by_hour[i][station] = station_trips
        trip_count[i] += total_trips

In [ ]:
transitions = []
total_total = 0

total_low = 0
total_mid = 0   
total_high = 0

total_num_low = 0
total_num_mid = 0
total_num_high = 0

for hour in stations_transition_by_hour:
    num_stations = len(stations_transition_by_hour[hour])
    total_hour = 0
    
    hour_low = 0
    hour_mid = 0   
    hour_high = 0
    
    num_low = 0
    num_mid = 0
    num_high = 0
    
    num_unique_stations = len(stations_transition_by_hour[hour])
    for station in stations_transition_by_hour[hour]:
        num = stations_transition_by_hour[hour][station]
        total_hour += num
        if  num/trip_count[hour] < 1/(3*num_unique_stations):
            num_low +=1
            hour_low += num
        elif num/trip_count[hour] < 1/(2*num_unique_stations):
            num_mid +=1
            hour_mid += num
        else:
            num_high += 1
            hour_high += num
    if total_hour == 0:
        print(f'No trips detected in {hour}')
        continue
    print(f'In hour: {hour} there was {total_hour} observations and {num_unique_stations} unique stations')
    print(f'Low: {num_low} stations, Mid: {num_mid} stations, High: {num_high} stations')
    print(f'Low: {hour_low/total_hour}, Mid: {hour_mid/total_hour}, High: {hour_high/total_hour}')
    print()
    
    total_total += total_hour
    
    total_low += hour_low
    total_mid += hour_mid
    total_high += hour_high
    
    total_num_low += num_low
    total_num_mid += num_mid
    total_num_high += num_high

print()
print(f'Over the entire period: {start_time} to {end_time} there was {total_total} observations and {total_unique_stations} total unique stations')
print(f'Low: {total_low/total_total}, Mid: {total_mid/total_total}, High: {total_high/total_total}') 

In [ ]:
transitions